In [ ]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'calibrate.ipynb'

from ionpy.util import Config

# Setup direcrtories
root = Path("/storage/vbutoi/scratch/ESE")
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [ ]:
%%yaml default_cfg

# We can change the lr and weight decay mid run.
# Usually we are going to load the old optimizer state.
# optim:
#   weight_decay: 0.0 
#   lr: 1.0e-4

# Optionally we can change the loss function.
# loss_func: 
#   _class: '?'
#   from_logits: True
#   batch_reduction: 'mean' 

train:
  epochs: '?' # How much longer to train for.
  load_chkpt: '?' # Which model do we load
  pretrained_dir: '?' # Which runs are we restarting

In [ ]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.callbacks.ShowPredictions
  epoch:
    - ese.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

In [ ]:
%%yaml experiment_cfg 

name: "OCTA_FULLRES_LTS_LongRuns_RESTARTED" # We will treat this as a NEW EXPERIMENT.

train:
    epochs: 100
    load_chkpt: 'last'
    pretrained_dir: 
        - "/storage/vbutoi/scratch/ESE/training/08_07_24_OCTA_FULLRES_SoftDice"
        - "/storage/vbutoi/scratch/ESE/training/08_07_24_OCTA_FULLRES_CrossEntropy"

In [ ]:
from ese.analysis.analysis_utils.submit_utils import get_ese_restart_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([callbacks_cfg])

restart_cfgs = get_ese_restart_configs(
    exp_cfg=experiment_cfg,
    base_cfg=base_cfg
)

In [ ]:
len(restart_cfgs)

## Running Jobs

In [ ]:
from ese.experiment import run_ese_exp, submit_ese_exps, CalibrationExperiment

In [ ]:
# ####### Run individual jobs
# run_ese_exp(
#     config=restart_cfgs[0], 
#     experiment_class=CalibrationExperiment,
#     run_name='debug',
#     show_examples=True,
#     track_wandb=False,
#     gpu='0',
#     # gpu='4',
# )

In [ ]:
#### Run Batch Jobs
submit_ese_exps(
    config_list=cal_cfgs,
    experiment_class=PostHocExperiment,
    track_wandb=True,
    available_gpus=['0', '1', '2', '3']
    # available_gpus=['4', '5', '6', '7']
)